In [ ]:
import pandas as pd
import yaml
import numpy as np
import scipy.ndimage
import time, os, sys
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
from cellpose import utils
from matplotlib import cm
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops, regionprops_table
import math
from scipy.ndimage import gaussian_filter
import sys
from cellpose import models, io, plot
import os
import glob

In [ ]:
sys.path.append("../feature_extraction/")
import functions

In [ ]:
parameter_file = "../feature_extraction/local/parameters_olya_test.yml"
parameters = functions.read_parameters(parameter_file)

In [ ]:
input_path = parameters["input_path"]

# All files ending with .txt
file_list = glob.glob(input_path + "*.tif")
#print(glob.glob(input_path + "*.tif")) 
for filename in file_list:
    #print(filename)
    print(filename.split("/")[-1].split(".")[0])

In [ ]:
merged_properties_df = pd.DataFrame()

for ind, filepath in enumerate(file_list):
    
    filename = filepath.split("/")[-1]
    
    img = functions.read_image(parameters,filepath)
    img_seg = functions.get_image_for_segmentation(parameters,img)
    
    cellpose_mask = functions.get_cellpose_segmentation(parameters, img_seg)
    #nuclei_mask = functions.get_nuclei_mask(parameters, img, cellpose_mask)
    #golgi_mask = functions.get_golgi_mask(parameters, img, cellpose_mask)
    properties_df = functions.get_features_from_cellpose_seg(parameters, img, cellpose_mask, filename)
    
    
    
    print(properties_df.head())
    #properties_df.to_csv("image_%s.csv" % ind)
    properties_df.to_csv(filename.split(".")[0] + ".csv")
    
    if len(merged_properties_df.index) < 10:
        merged_properties_df = properties_df.copy()
    else:
        merged_properties_df.append(properties_df, ignore_index=True)
        
merged_properties_df.to_csv("merged.csv")

In [ ]:
filename = parameters['image_path']
img = skimage.io.imread(filename)
output_path = parameters['output_path']
output_filename = parameters["output_filename"]
output_filepath = output_path + output_filename

# extract channels

im_junction = img[:,:,parameters["channel_junction"]]
im_nucleus = img[:,:,parameters["channel_nucleus"]]
im_golgi = img[:,:,parameters["channel_golgi"]]
if parameters["channel_expression_marker"] >= 0:
    im_marker = img[:,:,parameters["channel_expression_marker"]]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(im_junction, cmap=plt.cm.gray)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(im_nucleus, cmap=plt.cm.gray)

In [ ]:
im_seg = np.array([im_junction, im_nucleus])


model = models.Cellpose(gpu=True, model_type='cyto')

channels = [1,2]

masks, flows, styles, diams = model.eval(im_seg, diameter=parameters["estimated_cell_diameter"], channels=channels)

io.masks_flows_to_seg(im_seg , masks, flows, diams, output_filename, channels)


In [ ]:
print(diams)
flows[2].shape

In [ ]:
fig, ax = plt.subplots(1,3)
ax[0].imshow(im_junction, cmap=plt.cm.gray)
ax[2].imshow(flows[2], cmap=plt.cm.gray)
ax[1].imshow(im_nucleus, cmap=plt.cm.gray)
#ax[1].imshow(flows[2], cmap=plt.cm.gray)

In [ ]:
# DISPLAY RESULTS
from cellpose import plot

fig = plt.figure(figsize=(12,5))
plot.show_segmentation(fig, im_seg, masks, flows[0], channels=channels)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.imshow(masks)